In [1]:
import openpyxl # 导入openpyxl模块
wb = openpyxl.load_workbook('a.xlsx') # 创建workbook对象
ws = wb.active

In [2]:
alphabets = [
            "{",                "}",                "if",               "(",                ")",
            "then",             "else",             "while",            "ID",               "=",
            ">",                "<",                ">=",               "<=",               "==",
            "+",                "-",                "*",                "/",                "NUM",
            "E",                "$",                ";",                "cs",     "s",
            "ifs",           "whiles",        "as",         "ss",            "boole",
            "ae",        "boolop",           "me",         "aep",   "se",
            "mep",    "program"
]

In [3]:
seq = []
for cell in ws["2"]:
    if cell.value is not None:
        for i in range(0,36):
            if alphabets[i] == cell.value:
                seq.append(i)
print(seq)
print(len(seq))

[0, 2, 3, 4, 5, 6, 7, 1, 8, 9, 11, 10, 13, 12, 14, 15, 16, 17, 18, 19, 22, 21, 23, 24, 25, 26, 27, 23, 28, 29, 30, 31, 32, 33, 34, 35]
36


In [4]:
# write
with  open('b.txt','w') as f:
    i = 0;
    for row in ws['3:59']: #for each row
        f.write('L = new LRState();\n')
        f.write('//state{0}'.format(i))
        f.write("\n")
        i += 1;
        for cell in row[1:]:
            if cell.value is None:
                continue
            if str(cell.value)[0] == 's':
                f.write('L.add(0,0,{0},{1});\n'.format(seq[cell.column - 2],cell.value[1:]))
            elif str(cell.value)[0] == 'r':
                f.write('L.add(0,1,{0},{1});\n'.format(seq[cell.column - 2],cell.value[1:]))
            elif str(cell.value)[0] == 'a':
                f.write('L.add(0,2,21,0);\n')
            else:
                f.write('L.add(1,0,{0},{1});\n'.format(seq[cell.column - 2],cell.value))
        f.write('LRStates.add(L);\n')
    f.close()

In [5]:
cell = ws['8'][2]
print(str(cell.value)[0] == 's')

True
